This is an example notebook on how to use NAM library for shape matching

In [1]:
# import libraries
import sys 
sys.path.append('..')
from model.neural_adjoint_map import *
from model.optimizer import *


In [2]:
#load data
import yaml
sys.path.append('./../../NRP/Unsupervised-Learning-of-Robust-Spectral-Shape-Matching/')
import logging
from os import path as osp

from datasets import build_dataset
from models import build_model

import yaml
from utils.tensor_util import to_device,to_numpy
import torch
import numpy as np

#torch.cuda.set_device(0)

#pointcloud

DATASET='smal'

with open('./../../NRP/Unsupervised-Learning-of-Robust-Spectral-Shape-Matching/options/test/'+DATASET+'.yaml', 'r') as f:
    opt = yaml.load(f, Loader=yaml.SafeLoader)
opt['is_train']=False
test_set = build_dataset(opt['datasets']['test_dataset'])

shape1=test_set[3]['first']
shape2=test_set[3]['second']

In [3]:
from methods.linear_ref import *
from methods.nam_ref import *


In [ ]:

# Load configuration
DATASET = 'smal'

test_set = build_dataset(opt['datasets']['test_dataset'])
opt['is_train'] = False
model = build_model(opt)
model.eval()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize variables
n_exp = 20
n_k = len(range(20, 201, 20)) + 1
mse_nzo = np.zeros((n_exp, n_k))
g_nzo = np.zeros((n_exp, n_k))

#ind=np.random.randint(0,len(test_set),n_exp)
ind=np.arange(n_exp)

results = {}

zo = []
abzo =[]
ima_l = []
fsf_l = []

n_zo = []
n_ima = []
n_fsf = []

from torch_cluster import nearest


for i in ind:
    data_x, data_y = to_device(test_set[i]['second'], device), to_device(test_set[i]['first'], device)
    if data_x['name'] != data_y['name']:
        dist_y = data_y['dist'].cpu()
        corr_x = data_x['corr']
        corr_y = data_y['corr']
        dist_y = to_numpy(dist_y)
        corr_x = to_numpy(corr_x)
        corr_y = to_numpy(corr_y)
        C = torch.linalg.pinv(data_y['evecs'][corr_y, :20]) @ data_x['evecs'][corr_x, :20]
        p2p_ini = nearest(data_x['evecs'][:, :20], data_y['evecs'][:, :20] @ C).cpu().numpy()

        p2p=fsf(data_x['evecs'],data_y['evecs'],p2p_ini,20,201,20).cpu()
        fsf_l.append(dist_y[p2p[corr_x],corr_y].mean())
        p2p=zoomout(data_x['evecs'],data_y['evecs'],p2p_ini,20,201,20).cpu()
        zo.append(dist_y[p2p[corr_x],corr_y].mean())
        p2p=ima(data_x['evecs'],data_y['evecs'],p2p_ini,20,201,20).cpu()
        ima_l.append(dist_y[p2p[corr_x],corr_y].mean())
        p2p=ab_zoomout(data_x['evecs'],data_y['evecs'],p2p_ini,20,201,20).cpu()
        abzo.append(dist_y[p2p[corr_x],corr_y].mean())
        p2p=neural_ima(data_x['evecs'],data_y['evecs'],p2p_ini,20,201,20).cpu()
        n_ima.append(dist_y[p2p[corr_x],corr_y].mean())
        p2p=neural_zoomout(data_x['evecs'],data_y['evecs'],p2p_ini,20,201,20).cpu()
        n_zo.append(dist_y[p2p[corr_x],corr_y].mean())
        p2p=neural_fsf(data_x['evecs'],data_y['evecs'],p2p_ini,20,201,20).cpu()
        n_fsf.append(dist_y[p2p[corr_x],corr_y].mean())

    i=i+1
    print(i)
print('fsf:',np.mean(fsf_l))
print('zo:',np.mean(zo))
print('ima:',np.mean(ima_l))
print('abzo:',np.mean(abzo))

print('n_fsf:',np.mean(n_fsf))
print('n_zo:',np.mean(n_zo))
print('n_ima:',np.mean(n_ima))



1
2
3
